# UDP통신
    
1. socket 이란?
   > IP와 포트를 활용 : 특정 포트에 연결되어 데이터를 보내거나 받고자 할 때  
   > <font color='red'>서버</font> : 서비스 주체로 클라이언트로부터 메시지 받고 응답한다.  
   > <font color='blue'>클라이언트</font> : 서비스 사용자  
2. UDP서버와 클라이언트를 만들어 보자
   
   

## 1. 사용중인 컴퓨터의 IP 확인

In [ ]:
!ipconfig 

## 2. UDP 서버
1. 주소 체계(family)
   > AF_INET : IPv4  
   > AF_INET6 : IPv6
2. 타입(type)
   > SOCK_STREAM : TCP  
   > SOCK_DGRAM : UDP

In [ ]:
#통신 모듈
import socket
import time
import threading
global gmsg  # main, thread 전역 변수
global tellomsg # Tello 상태 정보
global run_flag # thread 종료 변수

#주소 설정
tello_addr = ('0.0.0.0',38889) # Tello드론 38889
notebook_addr = ('0.0.0.0',38890) # 노트북 38890

# 메시지 송수신
def send(message,delay):
    try:
        sock.sendto(message.encode(),notebook_addr) # Tello드론->노트북
        print("보낸메시지:" + message)
    except Exception as e:
        print("[tello send]에러 메시지:" + str(e))
    #지연
    time.sleep(delay)        
# notebook -> tello드론 메시지 받기(recvfrom)
def receive():
    global gmsg
    global tellomsg
    global run_flag
    while run_flag:
        try:
            response, ip_addr = sock.recvfrom(1024) # 노트북 -> Tello드론                       
            gmsg = "[notebook]: %s, IP: %s" % (response.decode(encoding='UTF-8'), str(ip_addr))            
            print(gmsg)
            # (Echo 서비스) tello정보 보내기 
            send("[tello응답] %s ok" % response.decode(encoding='UTF-8'),1)
        except Exception as e:
            print("[tello receive] 에러 메시지:" + str(e))
            break
            
# Tello드론(tello_addr) 통신포트 바인딩
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(tello_addr)

# 노트북으로부터 메시지 받기 쓰레드 생성 및 시작
run_flag = True
recvThread = threading.Thread(target=receive)
recvThread.daemon = True
recvThread.start()            

# 통신 시작
while True:
    try:
        print('■ Tello드론서버는 대기모드')
        message = input()
        if 'quit' in message:
            print('프로그램 종료')
            run_flag = False
            recvThread.join()
            sock.close()
            break        
        else:
            send(gmsg,3)
    except KeyboardInterrupt as e:
        sock.close()
        break

■ Tello드론서버는 대기모드
[notebook]: 반갑습니다. !!, IP: ('168.131.49.99', 60572)
[tello send]에러 메시지:[WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다
